In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

import warnings
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv())  # read the .env file in local directory
openai.api_key = os.environ[
    "OPENAI_API_KEY"
]  # to use this notebook we must set up an API key from OpenAI

llm_model = "gpt-3.5-turbo-0125" 

In [2]:
from langchain_openai import ChatOpenAI # used to interact openai chat models
from langchain.chains import ConversationChain # this allows us to create conversational chat pipeline
from langchain.memory import ConversationBufferMemory # stores previous conversations

In [34]:
llm = ChatOpenAI() # we called it llm because we are going to use inside of conversation chain memory
memory = ConversationBufferMemory() # created buffer memory instance
conversation = ConversationChain( 
    llm=llm, 
    memory=memory,
    verbose=True # verbose=False means when we chat with the model model directly gives the answer we don't see internal process but if we change it =True
)

In [4]:
conversation.predict(input="Hi this is NoOne") # sending message to chat



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi this is NoOne
AI:

> Finished chain.


'Hello NoOne! How can I assist you today?'

In [5]:
conversation.predict(input="what is 8+7?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi this is NoOne
AI: Hello NoOne! How can I assist you today?
Human: what is 8+7?
AI:

> Finished chain.


'8 + 7 equals 15.'

In [6]:
conversation.predict(input="say my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi this is NoOne
AI: Hello NoOne! How can I assist you today?
Human: what is 8+7?
AI: 8 + 7 equals 15.
Human: say my name
AI:

> Finished chain.


'Your name is NoOne.'

In [7]:
print(memory.buffer) # we can see previous conversations 

Human: Hi this is NoOne
AI: Hello NoOne! How can I assist you today?
Human: what is 8+7?
AI: 8 + 7 equals 15.
Human: say my name
AI: Your name is NoOne.


In [8]:
memory.load_memory_variables({}) # laod_memory+variables expects input dictionary but it doesn't require any specific values

{'history': 'Human: Hi this is NoOne\nAI: Hello NoOne! How can I assist you today?\nHuman: what is 8+7?\nAI: 8 + 7 equals 15.\nHuman: say my name\nAI: Your name is NoOne.'}

In [9]:
memory.save_context({"input": "Hi, how are you"}, {"output": "Thanks for asking I'm fine, and you?"}) # entering a conversation by hand

In [10]:
print(memory.buffer)

Human: Hi this is NoOne
AI: Hello NoOne! How can I assist you today?
Human: what is 8+7?
AI: 8 + 7 equals 15.
Human: say my name
AI: Your name is NoOne.
Human: Hi, how are you
AI: Thanks for asking I'm fine, and you?


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi this is NoOne\nAI: Hello NoOne! How can I assist you today?\nHuman: what is 8+7?\nAI: 8 + 7 equals 15.\nHuman: say my name\nAI: Your name is NoOne.\nHuman: Hi, how are you\nAI: Thanks for asking I'm fine, and you?"}

In [12]:
# When creating chatbots or memory needed services bot have to remember the conversation that's done. 
# But meanwhile remembering the whole conversation would cost much (next token prediction happens based on all previous tokens)
# So overcome this issue langchain provides window buffer memory that will store windows of memory

from langchain.memory import ConversationBufferWindowMemory

In [35]:
memory = ConversationBufferWindowMemory(k=1) # k=1 is our window length that means it'll remember just previous message

In [14]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [15]:
memory.load_memory_variables({})

# as we can see it doesn't remember first message

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [16]:
llm = ChatOpenAI() # we called it llm because we are going to use inside of conversation chain memory
memory = ConversationBufferWindowMemory(k=1) # created buffer memory instance
conversation = ConversationChain( 
    llm=llm, 
    memory=memory,
    verbose=False # verbose=False means when we chat with the model model directly gives the answer we don't see internal process but if we change it =True
)

In [17]:
conversation.predict(input="Hi this is NoOne")

'Hello, NoOne! How can I assist you today?'

In [18]:
conversation.predict(input="When did US send rocket to Moon?")

'The United States sent the Apollo 11 mission to the Moon on July 16, 1969. The mission successfully landed on the Moon on July 20, 1969, with astronaut Neil Armstrong becoming the first person to walk on the lunar surface.'

In [19]:
conversation.predict(input="Say my name")

"I'm sorry, I do not have access to personal information such as your name."

In [20]:
# ConversationTokenBufferMemory is same but limit is determined by token count

from langchain.memory import ConversationTokenBufferMemory

llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [36]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50) # when limit is reached it takes into account last conversations
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [22]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [23]:
# !pip install tiktoken

In [24]:
import tiktoken # tiktoken is tokenization library by OpenAI

In [25]:
# encoding = tiktoken.get_encoding("cl100k_base") # this is the tokenizer for gpt3.5-turbo also the line below works
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125") # give model name and it chooses right encoding 

In [26]:
encoded_conv = encoding.encode(memory.load_memory_variables({})["history"]) # encoding conversation history

In [27]:
[encoding.decode_single_token_bytes(token) for token in encoded_conv]

[b'Human',
 b':',
 b' AI',
 b' is',
 b' what',
 b'?',
 b'!\n',
 b'AI',
 b':',
 b' Amazing',
 b'!\n',
 b'Human',
 b':',
 b' Back',
 b'prop',
 b'agation',
 b' is',
 b' what',
 b'?\n',
 b'AI',
 b':',
 b' Beautiful',
 b'!\n',
 b'Human',
 b':',
 b' Chat',
 b'bots',
 b' are',
 b' what',
 b'?\n',
 b'AI',
 b':',
 b' Ch',
 b'arming',
 b'!']

In [28]:
len(encoded_conv)

35

In [29]:
# with conversation summary buffer memory LLM creates a summary and continues based on that
from langchain.memory import ConversationSummaryBufferMemory

In [37]:
schedule = "Your day kicks off at 7:30 AM with a strategy meeting with the marketing team—bring your notes on the new campaign.\
At 10 AM, you have a deep dive into model optimization, so set up your environment beforehand.\
From 1 PM to 3 PM, you'll be reviewing code for the fraud detection project, making sure everything is on track.\
At 4 PM, there's a brainstorming session on future AI integrations, followed by a quick catch-up with your mentor at 6 PM.\
Wrap up the day with some quiet time to reflect on new ideas before heading home."
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [31]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the day's schedule. The AI outlines a busy day filled with meetings, code reviews, and brainstorming sessions."}

In [32]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [33]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule. The AI outlines a busy day filled with meetings, code reviews, and brainstorming sessions.
Human: What would be a good demo to show?
AI:

> Finished chain.


"A good demo to show could be a new feature or functionality that has been recently developed and is ready for testing. It could showcase the user interface design, the seamless integration with existing systems, or the improved performance metrics. Another option could be a prototype of a future project to gather feedback and generate excitement among stakeholders. Ultimately, the demo should highlight the team's progress and innovation in a visually engaging and informative way."